In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from pathlib import Path

BASE_URL = "https://www.myjobmag.co.ke/search/jobs"
QUERY = "developer"  # you can change this to other keywords like 'python', 'data', etc.
OUTPUT_EXCEL = Path("myjobmag_developer_jobs.xlsx")



In [6]:
def fetch_search_page(page: int = 1, query: str = QUERY) -> BeautifulSoup:
    """Fetch a single search results page and return its BeautifulSoup object."""
    params = {"q": query}
    if page > 1:
        params["page"] = page

    resp = requests.get(BASE_URL, params=params, timeout=30)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "html.parser")


def parse_jobs_from_soup(soup: BeautifulSoup) -> list[dict]:
    """Parse job cards from a search results page into a list of dicts."""
    jobs = []

    # Job cards appear as list items under the results section. We look for blocks with job titles like 'DevOps Developer at Davis & Shirtliff Group'.
    results_container = soup.find("div", class_="listings") or soup  # fallback to whole soup if structure changes

    # A robust pattern is to look for <h2> or <h3> elements that contain the job title and link.
    for header in results_container.find_all(["h2", "h3"]):
        a = header.find("a")
        if not a:
            continue

        title_text = a.get_text(strip=True)
        job_url = a.get("href")
        if job_url and job_url.startswith("/"):
            job_url = f"https://www.myjobmag.co.ke{job_url}"

        # Company name often appears near the header, for example the previous or parent element
        company = None
        parent = header.parent
        if parent:
            company_el = parent.find("a", class_="company-name") or parent.find("span", class_="company")
            if company_el:
                company = company_el.get_text(strip=True)

        # Short description / snippet
        summary = None
        desc_el = parent.find("p") if parent else None
        if desc_el:
            summary = desc_el.get_text(" ", strip=True)

        # Date – often shown in a small tag or span with the date text like '05 January'
        date_posted = None
        date_el = parent.find("span", class_="job-date") or parent.find("time")
        if date_el:
            date_posted = date_el.get_text(strip=True)

        jobs.append({
            "title": title_text,
            "company": company,
            "summary": summary,
            "date_posted_text": date_posted,
            "job_url": job_url,
        })

    return jobs


def scrape_all_pages(max_pages: int | None = None) -> list[dict]:
    """Scrape all result pages until there are no new jobs or max_pages is reached."""
    all_jobs: list[dict] = []
    page = 1

    while True:
        print(f"Scraping page {page}...")
        soup = fetch_search_page(page)
        jobs = parse_jobs_from_soup(soup)

        if not jobs:
            print("No jobs found on this page. Stopping.")
            break

        all_jobs.extend(jobs)

        # Stop if there is no 'next' button or we reached max_pages
        pagination = soup.find("ul", class_="pagination") or soup.find("div", class_="pagination")
        has_next = pagination and ("Next" in pagination.get_text())

        if max_pages is not None and page >= max_pages:
            print("Reached max_pages limit. Stopping.")
            break
        if not has_next:
            print("No 'Next' button found. Assuming last page.")
            break

        page += 1

    return all_jobs



In [7]:
def normalize_jobs(jobs: list[dict]) -> pd.DataFrame:
    """Convert list of job dicts into a clean DataFrame with extra metadata."""
    df = pd.DataFrame(jobs)
    if df.empty:
        return df

    # Add scrape metadata
    df["scrape_timestamp"] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

    # Basic cleanup
    df["title"] = df["title"].astype(str).str.strip()
    df["company"] = df["company"].astype(str).str.strip()
    df["job_url"] = df["job_url"].astype(str).str.strip()

    # Create a simple unique key for de-duplication
    df["job_key"] = (df["title"] + " | " + df["company"] + " | " + df["job_url"]).str.lower()
    return df


def merge_with_existing(df_new: pd.DataFrame, output_path: Path = OUTPUT_EXCEL) -> pd.DataFrame:
    """Merge new scraped data with existing Excel file, avoiding duplicates."""
    if output_path.exists():
        existing = pd.read_excel(output_path)
        if "job_key" not in existing.columns:
            # Rebuild job_key if missing
            existing["title"] = existing["title"].astype(str)
            existing["company"] = existing["company"].astype(str)
            existing["job_url"] = existing["job_url"].astype(str)
            existing["job_key"] = (existing["title"] + " | " + existing["company"] + " | " + existing["job_url"]).str.lower()
        combined = pd.concat([existing, df_new], ignore_index=True)
        combined = combined.drop_duplicates(subset=["job_key"]).reset_index(drop=True)
        return combined
    else:
        return df_new


def save_to_excel(df: pd.DataFrame, output_path: Path = OUTPUT_EXCEL) -> None:
    """Save the DataFrame to an Excel file."""
    if df.empty:
        print("No data to save.")
        return

    df.to_excel(output_path, index=False)
    print(f"Saved {len(df)} jobs to {output_path}")



In [8]:
# Run the full scraping pipeline

jobs_raw = scrape_all_pages()  # you can pass max_pages=2 for quick tests
print(f"Scraped {len(jobs_raw)} raw jobs.")

jobs_df = normalize_jobs(jobs_raw)
print(f"After normalization: {len(jobs_df)} jobs.")

merged_df = merge_with_existing(jobs_df)
print(f"After merging with existing Excel: {len(merged_df)} unique jobs.")

save_to_excel(merged_df)

merged_df.head()


Scraping page 1...
No 'Next' button found. Assuming last page.
Scraped 18 raw jobs.
After normalization: 18 jobs.
After merging with existing Excel: 18 unique jobs.


C:\Users\dkyalo\AppData\Local\Temp\ipykernel_80176\2656231451.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df["scrape_timestamp"] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")


Saved 18 jobs to myjobmag_developer_jobs.xlsx


,title,company,summary,date_posted_text,job_url,scrape_timestamp,job_key
0,DevOps Developer at Davis & Shirtliff Group,None,None,None,https://www.myjobmag.co.ke/job/devops-develope...,2026-01-07 07:39:20,devops developer at davis & shirtliff group | ...
1,Software Developer Attaché (3 Months Contract)...,None,None,None,https://www.myjobmag.co.ke/job/software-develo...,2026-01-07 07:39:20,software developer attaché (3 months contract)...
2,Software Developer - 6 Posts at University of ...,None,None,None,https://www.myjobmag.co.ke/job/software-develo...,2026-01-07 07:39:20,software developer - 6 posts at university of ...
3,Account Developer at The Coca-Cola Company,None,None,None,https://www.myjobmag.co.ke/job/account-develop...,2026-01-07 07:39:20,account developer at the coca-cola company | n...
4,ICT Senior Officer – Data Architect and Develo...,None,None,None,https://www.myjobmag.co.ke/job/ict-senior-offi...,2026-01-07 07:39:20,ict senior officer – data architect and develo...


## How to run this notebook every 24 hours

1. Open this notebook in Jupyter (or VS Code / Cursor) and run all cells. This will:
   - Scrape all `developer` jobs from [`https://www.myjobmag.co.ke/search/jobs?q=developer`](https://www.myjobmag.co.ke/search/jobs?q=developer).
   - Merge with the existing `myjobmag_developer_jobs.xlsx` file (if it exists).
   - Save an updated Excel file with all unique jobs.

2. To automate it every 24 hours on Windows (simple approach):
   - Install Jupyter and ensure `python` is in your PATH.
   - Create a `.bat` file that runs:

   ```bat
   cd /d C:\Users\dkyalo\Desktop\Trial
   jupyter nbconvert --to notebook --execute jobs_scraper.ipynb --output jobs_scraper_executed.ipynb
   ```

   - Use **Task Scheduler** to create a basic task that runs this `.bat` file once every day.

3. Each daily run will append new jobs (if any) and avoid duplicates using the `job_key` column.

